In [ ]:
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import pandas as pd
import sqlite3
import time
import os


# 🔧 Pad naar database en timeout
DB_PATH = 'C:/Users/NihalBoukhoubza/OneDrive/Scriptie/shoppingDB_sqlite.sqlite'
TIMEOUT = 5

# ✅ Functie om query te checken
def check_query(args):
    query, i, queryNumber, CORRECT_RESULTS = args
    message = ""

    try:
        # Verbind binnen de thread
        conn = sqlite3.connect(DB_PATH, timeout=1.0)
        start = time.time()
        answer = pd.read_sql_query(query, conn)
        conn.close()
        end = time.time()
        processtime = end - start

        correct = CORRECT_RESULTS.get(queryNumber)
        score = 0
        if correct is not None:
            for item in correct:
                if answer.equals(item):
                    score = 2
                    break

        return [score, message, processtime]
    except Exception as e:
        return [1, str(e), 20]

# ✅ Correctheid berekenen met threads en logging
def calculateCorrectness(queryList, queryNumberList):
    result, error, processtime = [], [], []

    with ThreadPoolExecutor(max_workers=22) as executor:
        futures = [executor.submit(check_query, (queryList[i], i, queryNumberList[i], CORRECT_RESULTS))
                   for i in range(len(queryList))]

        for i, future in enumerate(futures):
            try:
                item = future.result(timeout=TIMEOUT)
                result.append(item[0])
                error.append(item[1])
                processtime.append(item[2])
            except TimeoutError:
                print(f"⚠️ Query {i} duurde langer dan {TIMEOUT} seconden.")
                result.append(3)
                error.append("timeout")
                processtime.append(20)
            except Exception as e:
                print(f"❌ Fout bij query {i}: {str(e)}")
                result.append(1)
                error.append(str(e))
                processtime.append(20)

            if i % 100 == 0 and i > 0:
                print(f"✅ {i} queries verwerkt...")

    return result, error, processtime

# ✅ Startpunt script
if __name__ == '__main__':
    print("📥 Verbinden met database...")
    conn = sqlite3.connect(DB_PATH)

    CORRECT_RESULTS = {
    '3_3': #Select all different names and ids of the customers who never made a purchase at a store with the name ‘Kumar’. 
        # Also include the customers who never made a purchase, but are still represented in the customer relation of our database.
       [pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Kumar'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Coop'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Hoogvliet'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Jumbo'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Sligro'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Albert Hein'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Lidl'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Dirk'
                                )
                            ''', conn)

                ],
    '3_4': #Select all different names and ids of customers who made at least one purchase at a store with the name ‘Kumar’ and never a purchase at a store with a different name than ‘Kumar’.        
        [pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Kumar' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Kumar'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Jumbo' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Jumbo'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Dirk' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Dirk'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Hoogvliet' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Hoogvliet'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Albert Hein' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Albert Hein'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Sligro' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Sligro'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Coop' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Coop'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Lidl' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Lidl'
                            )
                            ''', conn)
                ],
    '3_2': #Select all distinct combinations of names and ids of customers who have both a shoppinglist and a purchase, both for the same date in 2018.
        [pd.read_sql_query('''SELECT DISTINCT c.cID, c.cName
                            FROM customer as c, shoppinglist as sl, purchase as p
                            WHERE c.cID = sl.cID
                            AND p.cID = c.cID
                            and sl.date = p.date
                            and sl.date LIKE'%2018%';
                            ''', conn)
                ],
    '4_3': #Write a SQL query for the following RA query
        [pd.read_sql_query('''select sName
                            from store
                            except
                            select sName
                            from (
                               select s.sName, c.city
                                from store as s, (
                                    select city from customer
                                    union
                                    select city from store) as c
                                except
                                select sName, city from store) as a;
                            ''', conn)     
                ],
    '4_4': #List the names of all of those customers, and only of those customers, 
        #that spent an amount of money on any one date that is at least 75% of the maximal amount of money ever spent by a single customer on a single day.
        [pd.read_sql_query('''with ods(cID, date, amount) as (   
                                    select cID, date, sum(quantity * price)   
                                    from purchase   
                                    group by cID, date),
                                mods(amount) as (   
                                    select max(amount)
                                    from ods)
                                select cName from customer, ods, mods
                                where customer.cID = ods.cID and     
                                ods.amount >= 0.75 * mods.amount;
                            ''', conn)
                ],
    '4_5':
        [pd.read_sql_query('''select x.city as city, sum(x.numCustomersPerCity) as numCustomersPerCity
                            from (
                            select y.city, count(*) as numCustomersPerCity
                            from (
                                select distinct c.city, c.cid
                                from customer as c, store as s, purchase as p
                                where c.cid = p.cid and s.sid = p.sid and s.city = "Eindhoven"
                            ) as y
                            group by y.city
                            union
                            select city, 0 as numCustomersPerCity
                            from store
                            union
                            select city, 0 as numCustomersPerCity
                            from customer
                            ) as x
                            group by x.city;''', conn)]
    }


    # 📂 Laad de query data
    filename = "C:/Users/NihalBoukhoubza/OneDrive/Scriptie/query_analysis_with_time.csv"
    data = pd.read_csv(filename, sep=',', engine='python', header=0)
    data.columns = data.columns.str.strip()

    print("🔍 Kolommen:", data.columns)

    # 🧪 Optioneel: tijdelijk beperken tot eerste 100 queries voor test
    # data = data.head(100)

    print("🚀 Queries controleren...")
    correctness = calculateCorrectness(data['Query'].tolist(), data['Query Name'].tolist())
    data['correct'] = correctness[0]
    data['errormessage'] = correctness[1]
    data['runningtime'] = correctness[2]

    # 💾 Opslaan
    output_file = "output_correct.csv"
    data.to_csv(output_file, sep='─', index=False, header=True)
    print(f"✅ Klaar! Resultaten opgeslagen in '{output_file}'")

📥 Verbinden met database...
🔍 Kolommen: Index(['Folder', 'File', 'Timestamp', 'Query Name', 'Query', 'Contains error',
       'Error Type', 'Error Message', 'Bracket Count', 'Table Count',
       'Where Count', 'Group By Count', 'Select Count', 'From Count',
       'Join Count', 'Having count', 'Distinct count', 'Group By & Having',
       'Order by count', 'Count Count', 'Sum Count', 'Max Count',
       'Limit Count', 'Union Count', 'As Count', '= Count', '< Count',
       '> Count', 'Like Count', 'In Count', 'with Count', 'Length Count',
       'Word Count', 'Subquery count', 'Comments Count',
       'Invalid Column Count', 'Invalid Tables Count', 'Where Condition Count',
       'Time Spent (seconds)', 'Time Spent (minutes)'],
      dtype='object')
🚀 Queries controleren...
⚠️ Query 8 duurde langer dan 5 seconden.
⚠️ Query 9 duurde langer dan 5 seconden.
⚠️ Query 11 duurde langer dan 5 seconden.
⚠️ Query 24 duurde langer dan 5 seconden.
⚠️ Query 27 duurde langer dan 5 seconden.
⚠️ Que

In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import sqlite3
import time
import os


# 🔧 Pad naar database en timeout
DB_PATH = 'C:/Users/NihalBoukhoubza/OneDrive/Scriptie/shoppingDB_sqlite.sqlite'
MAX_WORKERS = 22
BATCH_SIZE = 100 

# ✅ Functie om query te checken
def check_query(args):
    query, i, queryNumber, CORRECT_RESULTS = args
    message = ""
    try:
        conn = sqlite3.connect(DB_PATH, timeout=1.0)
        start = time.time()
        answer = pd.read_sql_query(query, conn)
        conn.close()
        end = time.time()
        processtime = end - start

        correct = CORRECT_RESULTS.get(queryNumber)
        score = 0
        if correct:
            for item in correct:
                if answer.equals(item):
                    score = 2
                    break

        return i, [score, message, processtime]
    except Exception as e:
        return i, [1, str(e), 20]

# ✅ Correctheid berekenen met threads en logging
def calculateCorrectness(queryList, queryNumberList):
    result = [None] * len(queryList)
    error = [None] * len(queryList)
    processtime = [None] * len(queryList)

    for batch_start in range(0, len(queryList), BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, len(queryList))
        print(f"🔁 Batch {batch_start} - {batch_end} starten...")

        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = [
                executor.submit(check_query, (queryList[i], i, queryNumberList[i], CORRECT_RESULTS))
                for i in range(batch_start, batch_end)
            ]

            for completed in as_completed(futures):
                i, item = completed.result()
                result[i] = item[0]
                error[i] = item[1]
                processtime[i] = item[2]
                if i % 25 == 0:
                    print(f"✅ Query {i} verwerkt...")

    return result, error, processtime


# ✅ Startpunt script
if __name__ == '__main__':
    print("📥 Verbinden met database...")
    conn = sqlite3.connect(DB_PATH)

    CORRECT_RESULTS = {
    '3_3': #Select all different names and ids of the customers who never made a purchase at a store with the name ‘Kumar’. 
        # Also include the customers who never made a purchase, but are still represented in the customer relation of our database.
       [pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Kumar'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Coop'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Hoogvliet'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Jumbo'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Sligro'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Albert Hein'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Lidl'
                                )
                            ''', conn),
        pd.read_sql_query('''SELECT DISTINCT c.cName, c.cID
                            FROM customer c
                            WHERE c.cID NOT IN (
                                SELECT p.cID
                                FROM purchase p, store s
                                WHERE p.sID=s.sID AND s.sName='Dirk'
                                )
                            ''', conn)

                ],
    '3_4': #Select all different names and ids of customers who made at least one purchase at a store with the name ‘Kumar’ and never a purchase at a store with a different name than ‘Kumar’.        
        [pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Kumar' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Kumar'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Jumbo' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Jumbo'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Dirk' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Dirk'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Hoogvliet' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Hoogvliet'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Albert Hein' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Albert Hein'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Sligro' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Sligro'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Coop' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Coop'
                            )
                            ''', conn),
         pd.read_sql_query('''SELECT DISTINCT c1.cName, c1.cID
                            FROM purchase p1, store s1, customer c1
                            WHERE p1.sID=s1.sID AND s1.sName='Lidl' AND p1.cID=c1.cID AND c1.cID NOT IN (
                                SELECT p2.cID
                                FROM purchase p2, store s2
                                WHERE p2.sID=s2.sID AND s2.sName<>'Lidl'
                            )
                            ''', conn)
                ],
    '3_2': #Select all distinct combinations of names and ids of customers who have both a shoppinglist and a purchase, both for the same date in 2018.
        [pd.read_sql_query('''SELECT DISTINCT c.cID, c.cName
                            FROM customer as c, shoppinglist as sl, purchase as p
                            WHERE c.cID = sl.cID
                            AND p.cID = c.cID
                            and sl.date = p.date
                            and sl.date LIKE'%2018%';
                            ''', conn)
                ],
    '4_3': #Write a SQL query for the following RA query
        [pd.read_sql_query('''select sName
                            from store
                            except
                            select sName
                            from (
                               select s.sName, c.city
                                from store as s, (
                                    select city from customer
                                    union
                                    select city from store) as c
                                except
                                select sName, city from store) as a;
                            ''', conn)     
                ],
    '4_4': #List the names of all of those customers, and only of those customers, 
        #that spent an amount of money on any one date that is at least 75% of the maximal amount of money ever spent by a single customer on a single day.
        [pd.read_sql_query('''with ods(cID, date, amount) as (   
                                    select cID, date, sum(quantity * price)   
                                    from purchase   
                                    group by cID, date),
                                mods(amount) as (   
                                    select max(amount)
                                    from ods)
                                select cName from customer, ods, mods
                                where customer.cID = ods.cID and     
                                ods.amount >= 0.75 * mods.amount;
                            ''', conn)
                ],
    '4_5':
        [pd.read_sql_query('''select x.city as city, sum(x.numCustomersPerCity) as numCustomersPerCity
                            from (
                            select y.city, count(*) as numCustomersPerCity
                            from (
                                select distinct c.city, c.cid
                                from customer as c, store as s, purchase as p
                                where c.cid = p.cid and s.sid = p.sid and s.city = "Eindhoven"
                            ) as y
                            group by y.city
                            union
                            select city, 0 as numCustomersPerCity
                            from store
                            union
                            select city, 0 as numCustomersPerCity
                            from customer
                            ) as x
                            group by x.city;''', conn)]
    }


    # 📂 Laad de query data
    filename = "C:/Users/NihalBoukhoubza/OneDrive/Scriptie/query_analysis_with_time.csv"
    data = pd.read_csv(filename, sep=',', engine='python', header=0)
    data.columns = data.columns.str.strip()

    print("🔍 Kolommen:", data.columns)

    # 🧪 Optioneel: tijdelijk beperken tot eerste 100 queries voor test
    # data = data.head(100)

    print("🚀 Queries controleren...")
    correctness = calculateCorrectness(data['Query'].tolist(), data['Query Name'].tolist())
    data['correct'] = correctness[0]
    data['errormessage'] = correctness[1]
    data['runningtime'] = correctness[2]

    # 💾 Opslaan
    output_file = "output_correct.csv"
    data.to_csv(output_file, sep='─', index=False, header=True)
    print(f"✅ Klaar! Resultaten opgeslagen in '{output_file}'")
    

📥 Verbinden met database...
🔍 Kolommen: Index(['Folder', 'File', 'Timestamp', 'Query Name', 'Query', 'Contains error',
       'Error Type', 'Error Message', 'Bracket Count', 'Table Count',
       'Where Count', 'Group By Count', 'Select Count', 'From Count',
       'Join Count', 'Having count', 'Distinct count', 'Group By & Having',
       'Order by count', 'Count Count', 'Sum Count', 'Max Count',
       'Limit Count', 'Union Count', 'As Count', '= Count', '< Count',
       '> Count', 'Like Count', 'In Count', 'with Count', 'Length Count',
       'Word Count', 'Subquery count', 'Comments Count',
       'Invalid Column Count', 'Invalid Tables Count', 'Where Condition Count',
       'Time Spent (seconds)', 'Time Spent (minutes)'],
      dtype='object')
🚀 Queries controleren...
🔁 Batch 0 - 100 starten...
✅ Query 0 verwerkt...
✅ Query 50 verwerkt...
✅ Query 75 verwerkt...
✅ Query 25 verwerkt...
🔁 Batch 100 - 200 starten...
✅ Query 100 verwerkt...
✅ Query 125 verwerkt...
✅ Query 150 verwerk